In [1]:
import pandas as pd
import openai
import os

In [2]:
api_key = os.getenv("OPENAI_API_KEY")
print(type(api_key))
#print(api_key)

openai.api_key = api_key

<class 'str'>


In [6]:
data = pd.read_excel("data.xlsx")

In [7]:
labels = data['student'] 
texts = data['response']

df = pd.DataFrame(zip(texts, labels), columns = ['prompt','completion']) #[:300]
df.head()

,prompt,completion
0,\n\nColonizers or Victims?\n\nThroughout histo...,10th grade
1,\n\nColonizers or Victims?\n\nThe history of c...,college
2,\n\nRegression models are a common tool used b...,college
3,\n\nResidual Plots for Regression Model Valida...,10th grade
4,\n\nThe Mansfield Murders: A Case Study of a S...,college


In [8]:
df.to_json("essays.jsonl", orient='records', lines=True)

#### console commands

below lines are to be run in terminal. running inline with the '!' operator doesn't quite work as intended

In [ ]:
!openai tools fine_tunes.prepare_data -f essays.jsonl -q

In [ ]:
!openai api fine_tunes.create -t "essays_prepared_train.jsonl" -v "essays_prepared_valid.jsonl" --compute_classification_metrics --classification_positive_class " 10th grade" -m ada

In [10]:
# ran this one on curie

!openai api fine_tunes.results -i ft-cGUDQqtayDcpWvQ4tXxSKdhL > result.csv

In [32]:
# this one is running on ada

!openai api fine_tunes.results -i ft-z3lKpEXOcp5hwMlFs9sAm4h4 > result.csv

In [33]:
results = pd.read_csv('result.csv')
results[results['classification/accuracy'].notnull()].tail(1)
test = pd.read_json('essays_prepared_valid.jsonl', lines=True)
#print(test)
ft_model = 'ada:ft-personal-2023-03-13-22-18-36'

for i in range(len(test)):
    res = openai.Completion.create(model=ft_model, prompt=test['prompt'][i] + '\n\n###\n\n', max_tokens=1, temperature=0)
    test.loc[i, 'prediction'] = res['choices'][0]['text']

print(test)

                                              prompt   completion prediction
0  \n\nThe Inspiration for Sherlock Holmes\n\nAs ...      college         10
1  \n\nDuring World War II, the development of je...   10th grade    college
2  \n\nThe Secret to Knowing the Unknowable: Pred...      college    college
3  \n\nMikhael Subotzky’s Destructive Collage Pro...      college         10
4  \n\nProbability vs Likelihood\n\nProbability a...   10th grade    college
5  \n\nThe Roman gladiators were some of the most...   10th grade    college
6  \n\nVisualizing Overland Travel, When All Road...   10th grade    college
7  \n\nElections are a fundamental aspect of any ...   10th grade    college
